In [1]:
import flor

In [2]:
flor.dataframe("page_color")

,document_value,page,page_value,projid,tstamp,filename,page_color
0,6-Complaint__Judgment,1,0,pdf_parser,2024-03-06 09:14:11,flask,0
1,6-Complaint__Judgment,2,1,pdf_parser,2024-03-06 09:14:11,flask,0
2,6-Complaint__Judgment,3,2,pdf_parser,2024-03-06 09:14:11,flask,0
3,6-Complaint__Judgment,4,3,pdf_parser,2024-03-06 09:14:11,flask,0
4,6-Complaint__Judgment,5,4,pdf_parser,2024-03-06 09:14:11,flask,0
5,6-Complaint__Judgment,6,5,pdf_parser,2024-03-06 09:14:11,flask,0
6,6-Complaint__Judgment,7,6,pdf_parser,2024-03-06 09:14:11,flask,0
7,6-Complaint__Judgment,8,7,pdf_parser,2024-03-06 09:14:11,flask,0
8,6-Complaint__Judgment,9,8,pdf_parser,2024-03-06 09:14:11,flask,0
9,6-Complaint__Judgment,10,9,pdf_parser,2024-03-06 09:14:11,flask,0


In [3]:
flor.dataframe("first_page")

,document,document_value,page,page_value,projid,tstamp,filename,first_page
0,1,6-Complaint__Judgment,1,0,pdf_parser,2024-03-06 09:14:02,label_by_hand.py,1
1,1,6-Complaint__Judgment,2,1,pdf_parser,2024-03-06 09:14:02,label_by_hand.py,0
2,1,6-Complaint__Judgment,3,2,pdf_parser,2024-03-06 09:14:02,label_by_hand.py,0
3,1,6-Complaint__Judgment,4,3,pdf_parser,2024-03-06 09:14:02,label_by_hand.py,0
4,1,6-Complaint__Judgment,5,4,pdf_parser,2024-03-06 09:14:02,label_by_hand.py,0
...,...,...,...,...,...,...,...,...
229,6,george_santos_report,52,51,pdf_parser,2024-03-06 09:14:02,label_by_hand.py,0
230,6,george_santos_report,53,52,pdf_parser,2024-03-06 09:14:02,label_by_hand.py,0
231,6,george_santos_report,54,53,pdf_parser,2024-03-06 09:14:02,label_by_hand.py,0
232,6,george_santos_report,55,54,pdf_parser,2024-03-06 09:14:02,label_by_hand.py,0


In [4]:
flor.dataframe('merge-source', 'merged-text')

,document,document_value,page,page_value,projid,tstamp,filename,merge-source,merged-text
0,1,6-Complaint__Judgment,1,0,pdf_parser,2024-03-06 09:14:03,flask,ocr,JERMAINE LYONS + INTHE\n\n1306 Paddock Lane\n\...
1,1,6-Complaint__Judgment,2,1,pdf_parser,2024-03-06 09:14:03,flask,ocr,"2. That the Defendants, Sgt. Martin, Officer D..."
2,1,6-Complaint__Judgment,3,2,pdf_parser,2024-03-06 09:14:03,flask,ocr,COUNT TWO FALSE ARREST\n\ning allegations of t...
3,1,6-Complaint__Judgment,4,3,pdf_parser,2024-03-06 09:14:03,flask,ocr,"15, Asa proximate result of Defendant, Officer..."
4,1,6-Complaint__Judgment,5,4,pdf_parser,2024-03-06 09:14:03,flask,ocr,IVE\n\nCOUNT FI\nVIOLATION OF THE MARYLAND DEC...
...,...,...,...,...,...,...,...,...,...
463,6,george_santos_report,52,51,pdf_parser,2024-03-06 10:41:48,flask,txt,"allegation, finding that there was not substan..."
464,6,george_santos_report,53,52,pdf_parser,2024-03-06 10:41:48,flask,txt,status is stilllisted as\nactive\nCommittee st...
465,6,george_santos_report,54,53,pdf_parser,2024-03-06 10:41:48,flask,txt,"criminaldefense services, or whether the indiv..."
466,6,george_santos_report,55,54,pdf_parser,2024-03-06 10:41:48,flask,txt,significant charges ofviolations bythe Committ...
